In [1]:
text ='''
화려한 도시를 그리며 찾아왔네
그 곳은 춥고도 험한 곳
여기저기 헤매다 초라한 문턱에서
뜨거운 눈물을 먹는다
머나먼 길을 찾아 여기에 꿈을 찾아 여기에
괴롭고도 험한 이 길을 왔는데
이 세상 어디가 숲인지 어디가 늪인지
그 누구도 말을 않네
사람들은 저마다 고향을 찾아가네
나는 지금 홀로 남아서
빌딩 속을 헤매이다 초라한 골목에서
뜨거운 눈물을 먹는다
저기 저 별은 나의 마음을 알까 나의 꿈을 알까
괴로울 땐 슬픈 노래를 부른다
슬퍼질 땐 차라리 나 홀로 눈을 감고 싶어
고향의 향기 들으면서
저기 저 별은 나의 마음 알까 나의 꿈을 알까
괴로울 땐 슬픈 노래를 부른다
이 세상 어디가 숲인지 어디가 늪인지
그 누구도 말을 않네
슬퍼질 땐 차라리 나 홀로 눈을 감고 싶어
고향의 향기 들으면서
고향의 향기 들으면서
'''

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer          # 언어 토큰화
from tensorflow.keras.preprocessing.sequence import pad_sequences  # 인풋 데이터 길이 조절, 잘라내거나 붙이거나 해서 동일한 길이로 인풋시키는 용도
from tensorflow.keras.utils import to_categorical                  # 원핫인코딩을 위한 카테고리화 모듈

In [3]:
tokenizer = Tokenizer()                     # 토큰화 모듈 호출
tokenizer.fit_on_texts([text])              # 토큰화 모듈에 텍스트 투입
vocab_size = len(tokenizer.word_index) + 1  # 토큰화 모듈 길이에 1을 더하기. 
print('단어 집합의 크기 : %d' % vocab_size) # 단어집합 크기 65

단어 집합의 크기 : 65


In [5]:
sequences = list()                                      # 비어있는 리스트 출력
for line in text.split('\n'):                           # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]   # 문장을 숫자로 변환 인코딩
    for i in range(1, len(encoded)):                    # 인코딩 된 길이만큼 반복
        sequence = encoded[:i+1]                        # 시퀀스에 순서대로 투입
        sequences.append(sequence)                      # 시퀀시스 리스트에 시퀀스 추가
                                                      
print('학습에 사용할 샘플의 개수: %d' % len(sequences)) # 총 94개의 샘플

학습에 사용할 샘플의 개수: 94


In [6]:
max_len = max(len(l) for l in sequences)                # 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))          # 샘플의 최대 길이 9

샘플의 최대 길이 : 9


In [9]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre') # 시퀀스, 최대길이 9, 패딩은 오른쪽
sequences = np.array(sequences)                                     # 배열화하여 모델에 투입준비
X = sequences[:,:-1]                                                # 시퀀스 문제지              
y = sequences[:,-1]                                                 # 시퀀스 답지
y = to_categorical(y, num_classes=vocab_size)                       # 시퀀스 답지 원핫인코딩, 단어집합의 크기만큼 열 증가

In [10]:
from tensorflow.keras.models import Sequential              # 케라스 모델 호출
from tensorflow.keras.layers import Embedding, Dense, LSTM  # 임베딩, 풀리 커넥티드, LSTM

In [15]:
embedding_dim = 36                                     # 최초 임베딩 자료는 10차원으로 축소
hidden_units = 128                                     # LSTM히든레이어 출력 128

model = Sequential()                                   # 모델 제작
model.add(Embedding(vocab_size, embedding_dim))        # 65차원 투입, 10차원으로 전환
model.add(LSTM(hidden_units))                          # lstm 히든유닛은 128차원을 출력한다. 
model.add(Dense(vocab_size, activation='softmax'))     # 풀리 커넥티드, 출력 = 65개 중 한개, 소프트맥스 기준
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # 범주 교차 엔트로피, 아담, 정확도 기준
model.fit(X, y, epochs=300, verbose=1)                 # 모델 훈련. 에폭스 200, 안내 1번

Epoch 1/300
3/3 [==============================] - 1s 7ms/step - loss: 4.1747 - accuracy: 0.0532
Epoch 2/300
3/3 [==============================] - 0s 7ms/step - loss: 4.1658 - accuracy: 0.0745
Epoch 3/300
3/3 [==============================] - 0s 8ms/step - loss: 4.1576 - accuracy: 0.0957
Epoch 4/300
3/3 [==============================] - 0s 7ms/step - loss: 4.1475 - accuracy: 0.0957
Epoch 5/300
3/3 [==============================] - 0s 6ms/step - loss: 4.1346 - accuracy: 0.0745
Epoch 6/300
3/3 [==============================] - 0s 6ms/step - loss: 4.1174 - accuracy: 0.0638
Epoch 7/300
3/3 [==============================] - 0s 6ms/step - loss: 4.0891 - accuracy: 0.0638
Epoch 8/300
3/3 [==============================] - 0s 6ms/step - loss: 4.0448 - accuracy: 0.0638
Epoch 9/300
3/3 [==============================] - 0s 6ms/step - loss: 3.9744 - accuracy: 0.0426
Epoch 10/300
3/3 [==============================] - 0s 6ms/step - loss: 3.9106 - accuracy: 0.0426
Epoch 11/300
3/3 [===========

3/3 [==============================] - 0s 1ms/step - loss: 1.0578 - accuracy: 0.8404
Epoch 85/300
3/3 [==============================] - 0s 7ms/step - loss: 1.0436 - accuracy: 0.8404
Epoch 86/300
3/3 [==============================] - 0s 8ms/step - loss: 1.0318 - accuracy: 0.8617
Epoch 87/300
3/3 [==============================] - 0s 8ms/step - loss: 1.0091 - accuracy: 0.8723
Epoch 88/300
3/3 [==============================] - 0s 8ms/step - loss: 0.9899 - accuracy: 0.8404
Epoch 89/300
3/3 [==============================] - 0s 8ms/step - loss: 0.9603 - accuracy: 0.8723
Epoch 90/300
3/3 [==============================] - 0s 759us/step - loss: 0.9433 - accuracy: 0.8830
Epoch 91/300
3/3 [==============================] - 0s 7ms/step - loss: 0.9431 - accuracy: 0.8830
Epoch 92/300
3/3 [==============================] - 0s 8ms/step - loss: 0.9157 - accuracy: 0.8723
Epoch 93/300
3/3 [==============================] - 0s 8ms/step - loss: 0.8972 - accuracy: 0.8723
Epoch 94/300
3/3 [=============

3/3 [==============================] - 0s 6ms/step - loss: 0.2517 - accuracy: 0.9787
Epoch 167/300
3/3 [==============================] - 0s 8ms/step - loss: 0.2457 - accuracy: 0.9787
Epoch 168/300
3/3 [==============================] - 0s 7ms/step - loss: 0.2431 - accuracy: 0.9787
Epoch 169/300
3/3 [==============================] - 0s 9ms/step - loss: 0.2400 - accuracy: 0.9787
Epoch 170/300
3/3 [==============================] - 0s 2ms/step - loss: 0.2360 - accuracy: 0.9787
Epoch 171/300
3/3 [==============================] - 0s 258us/step - loss: 0.2317 - accuracy: 0.9787
Epoch 172/300
3/3 [==============================] - 0s 7ms/step - loss: 0.2291 - accuracy: 0.9787
Epoch 173/300
3/3 [==============================] - 0s 9ms/step - loss: 0.2264 - accuracy: 0.9787
Epoch 174/300
3/3 [==============================] - 0s 8ms/step - loss: 0.2219 - accuracy: 0.9681
Epoch 175/300
3/3 [==============================] - 0s 9ms/step - loss: 0.2194 - accuracy: 0.9787
Epoch 176/300
3/3 [===

3/3 [==============================] - 0s 8ms/step - loss: 0.0966 - accuracy: 0.9681
Epoch 249/300
3/3 [==============================] - 0s 3ms/step - loss: 0.0952 - accuracy: 0.9787
Epoch 250/300
3/3 [==============================] - 0s 240us/step - loss: 0.0949 - accuracy: 0.9787
Epoch 251/300
3/3 [==============================] - 0s 7ms/step - loss: 0.0942 - accuracy: 0.9681
Epoch 252/300
3/3 [==============================] - 0s 9ms/step - loss: 0.0939 - accuracy: 0.9787
Epoch 253/300
3/3 [==============================] - 0s 8ms/step - loss: 0.0932 - accuracy: 0.9787
Epoch 254/300
3/3 [==============================] - 0s 8ms/step - loss: 0.0923 - accuracy: 0.9787
Epoch 255/300
3/3 [==============================] - 0s 2ms/step - loss: 0.0918 - accuracy: 0.9787
Epoch 256/300
3/3 [==============================] - 0s 407us/step - loss: 0.0917 - accuracy: 0.9787
Epoch 257/300
3/3 [==============================] - 0s 8ms/step - loss: 0.0898 - accuracy: 0.9787
Epoch 258/300
3/3 [=

In [12]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word                                # 현재 단어를 init 단어에 할당
    sentence = ''                                           # 예측단어 문자로 할당

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]          # 텍스트를 투입하여 토큰화
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')# 토큰화한 텍스트숫자들을 시퀀스화(예 = [0,0,0,12,11])

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)         # 모델 적용
        result = np.argmax(result, axis=1)                 # 확률이 가장 높은 결과를 할당

        for word, index in tokenizer.word_index.items():  # 토큰화 단어목록을 for문으로 확인        
            if index == result:                           # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면          
                break                                     # 찾았음

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [19]:
print(sentence_generation(model, tokenizer, '화려한', 3))
print(sentence_generation(model, tokenizer, '그', 4))
print(sentence_generation(model, tokenizer, '여기저기', 3))
print(sentence_generation(model, tokenizer, '뜨거운', 2))
print(sentence_generation(model, tokenizer, '머나먼', 6))
print(sentence_generation(model, tokenizer, '괴롭고도', 4))
print(sentence_generation(model, tokenizer, '이', 5))
print(sentence_generation(model, tokenizer, '그', 3))
print(sentence_generation(model, tokenizer, '사람들은', 3))
print(sentence_generation(model, tokenizer, '나는', 3))
print(sentence_generation(model, tokenizer, '빌딩', 4))
print(sentence_generation(model, tokenizer, '뜨거운', 2))
print(sentence_generation(model, tokenizer, '저기', 8))
print(sentence_generation(model, tokenizer, '괴로울', 4))
print(sentence_generation(model, tokenizer, '슬퍼질', 7))
print(sentence_generation(model, tokenizer, '고향의', 2))
print(sentence_generation(model, tokenizer, '저기', 8))
print(sentence_generation(model, tokenizer, '괴로울', 4))
print(sentence_generation(model, tokenizer, '이', 5))
print(sentence_generation(model, tokenizer, '그', 3))
print(sentence_generation(model, tokenizer, '슬퍼질', 7))
print(sentence_generation(model, tokenizer, '고향의', 2))
print(sentence_generation(model, tokenizer, '고향의', 2))

화려한 도시를 그리며 찾아왔네
그 누구도 말을 않네 않네
여기저기 헤매다 초라한 문턱에서
뜨거운 눈물을 먹는다
머나먼 길을 찾아 여기에 꿈을 찾아 여기에
괴롭고도 험한 이 길을 왔는데
이 세상 어디가 숲인지 어디가 늪인지
그 누구도 말을 않네
사람들은 저마다 고향을 찾아가네
나는 지금 홀로 남아서
빌딩 속을 헤매이다 초라한 골목에서
뜨거운 눈물을 먹는다
저기 저 별은 나의 마음을 알까 나의 꿈을 알까
괴로울 땐 슬픈 노래를 부른다
슬퍼질 땐 차라리 나 홀로 눈을 감고 싶어
고향의 향기 들으면서
저기 저 별은 나의 마음을 알까 나의 꿈을 알까
괴로울 땐 슬픈 노래를 부른다
이 세상 어디가 숲인지 어디가 늪인지
그 누구도 말을 않네
슬퍼질 땐 차라리 나 홀로 눈을 감고 싶어
고향의 향기 들으면서
고향의 향기 들으면서


그곳은 땐 땐 슬픈


In [ ]:
화려한 도시를 그리며 찾아왔네
그 곳은 춥고도 험한 곳
여기저기 헤매다 초라한 문턱에서
뜨거운 눈물을 먹는다
머나먼 길을 찾아 여기에 꿈을 찾아 여기에
괴롭고도 험한 이 길을 왔는데
이 세상 어디가 숲인지 어디가 늪인지
그 누구도 말을 않네
사람들은 저마다 고향을 찾아가네
나는 지금 홀로 남아서
빌딩 속을 헤매이다 초라한 골목에서
뜨거운 눈물을 먹는다
저기 저 별은 나의 마음을 알까 나의 꿈을 알까
괴로울 땐 슬픈 노래를 부른다
슬퍼질 땐 차라리 나 홀로 눈을 감고 싶어
고향의 향기 들으면서
저기 저 별은 나의 마음 알까 나의 꿈을 알까
괴로울 땐 슬픈 노래를 부른다
이 세상 어디가 숲인지 어디가 늪인지
그 누구도 말을 않네
슬퍼질 땐 차라리 나 홀로 눈을 감고 싶어
고향의 향기 들으면서
고향의 향기 들으면서